# Despliegue

Vamos a realizar el despliegue, en donde a partir de nuestro modelo, se puede predecir con cierto grado de error establecido

In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
import sunpy.data.sample
import sunpy.map

import numpy as np
from ipywidgets import widgets
from IPython.display import display, Image, HTML

from ipywidgets  import interact, interactive, interact_manual , interactive_output, fixed

## Cargar Datos

In [2]:

column_names = ["Flare", "Date", "Start time", "Peak time", "End time", "Duration", "Peak counts", 
                    "Total counts", "Energy", "X pos", "Y pos", "Radial", "Active region", 
                    "Flag 1", "Flag 2", "Flag 3", "Flag 4", "Flag 5"]

df = pd.read_csv("hessi.solar.flare.UP_To_2018.csv",names=column_names, header = 0)
#df = pd.read_csv("hessi.solar.flare.UP_To_2018.csv",names=column_names, header = 0)

def preparar_caracteristicas(df):       
    # Deal with datetime format
    df['Start time'] = pd.to_datetime(df['Date'] + ' ' + df['Start time'])
    df['Peak time'] = pd.to_datetime(df['Date'] + ' ' + df['Peak time'])
    df['End time'] = pd.to_datetime(df['Date'] + ' ' + df['End time'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.loc[df['Start time'] >= df['End time'], 'End time'] += pd.Timedelta(days=1)
    df.loc[df['Start time'] >= df['Peak time'], 'Peak time'] += pd.Timedelta(days=1)
    
    df.sort_values(by='Start time', inplace=True)
    df['Time till next peak'] = (df['Peak time'].shift(-1) - df['Peak time']).dt.total_seconds()
    df = df.iloc[:-1]
    
    df['Solar cycle'] = ((df['Date'].dt.year - 2008) % 11) + 1
    
    split_values = df['Flag 5'].str.split(' ', expand=True).add_prefix('FlagN ')
    df = pd.concat([df, split_values], axis = 1)
    flags = df.filter(like = "Flag")
    flags = flags.drop("Flag 5", axis = 1)
    flag_dummies = pd.get_dummies(flags, prefix="Flag")
    flags = flag_dummies.groupby(flag_dummies.columns.str[-2:], axis=1).sum().astype(bool)
    flags["A"] = flags.apply(lambda x: "A0" if x["A0"] == 1 else ("A1" if x["A1"] == 1 else "A3"), axis=1)
    flags["Q"] = flags.apply(lambda x: "Q1" if x["Q1"] == 1 else ("Q2" if x["Q2"] == 1 else ("Q3" if x["Q3"] == 1 else ("Q4" if x["Q4"] == 1 else ("Q5" if x["Q5"] == 1 else ("Q6" if x["Q6"] == 1 else "Q7"))))), axis=1)
    flags.drop(columns=["A1", "A0", "A3","Q1", "Q2", "Q3", "Q4", "Q5", "Q6", "Q7"], inplace=True)
    df = pd.concat([df, flags], axis=1)
    df = df.filter(regex="^(?!.*Flag)")
    
    df[df["Flare"].duplicated(keep = False)]
    
    df.loc[:, "Flare number"] = df["Flare"].apply(lambda x: int(str(x)[-2:])) # Ver punto siguiente
    df["Time till peak"] = (df['Peak time'] - df['Start time']).dt.total_seconds()
    data = df[~(df["DF"] & df["DR"]) & ~df["NS"] & ~(df["GD"] & df["GE"] & df["GS"]) & ~df["PS"] & ~(df["EE"] & df["ES"]) & ~(df["SS"] & df["SD"] & df["SE"])]
    data.reset_index(drop=True, inplace=True)
    
    data = data[["Flare number", "Solar cycle", "Peak counts", "Total counts", "Energy", "X pos", "Y pos","Radial", "PE", "Duration", "Time till peak","Time till next peak"]]
    
    data.head()
    column = data[["Energy"]]
    order = ["3-6", "6-12","12-25", "25-50", "50-100", "100-300", "300-800", "800-7000", "7000-20000"]
    enc = OrdinalEncoder(categories=[order])
    label_encoding = enc.fit_transform(column)
    data["Energy"] = pd.DataFrame(label_encoding)
    
    return data

data = preparar_caracteristicas(df)
data.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_27736\746000528.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Solar cycle'] = ((df['Date'].dt.year - 2008) % 11) + 1
C:\Users\usuario\AppData\Local\Temp\ipykernel_27736\746000528.py:28: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  flags = flag_dummies.groupby(flag_dummies.columns.str[-2:], axis=1).sum().astype(bool)


,Flare number,Solar cycle,Peak counts,Total counts,Energy,X pos,Y pos,Radial,PE,Duration,Time till peak,Time till next peak
0,13,6,136,167304.0,2.0,592,-358,692,False,712,222.0,688.0
1,28,6,7,9504.0,1.0,604,-341,694,True,288,58.0,11388.0
2,32,6,15,11448.0,1.0,-310,375,487,False,216,90.0,12536.0
3,8,6,20,17400.0,2.0,-277,378,469,False,244,58.0,9684.0
4,10,6,336,313392.0,3.0,-272,390,476,True,236,82.0,240.0


In [3]:
def remover_outliers(data):
    data = data[data["Radial"]< 1865]
    data_outliers = data.copy()
    print(f"Shape of data : {data_outliers.shape}")
    print(f"Type of data : {type(data_outliers)}")
    # Create DBSCAN model with parameters
    model = DBSCAN(eps=300,min_samples=5) #### Estos parametros son los que se tendrían que ajustar 
    # (radio de cada anillo y número mínimo de muestras en el anillo para que sean válidos)
    # Fit model to data
    model.fit(data_outliers)
    # Printing total number of outliers
    print(f"Total number of outliers : {sum(model.labels_ == -1)}")
    # Outliers 
    outliers_index = model.labels_ == -1
    data_no_outliers = data_outliers[~outliers_index]
    data_no_outliers.reset_index(drop=True, inplace=True)
    data = data_no_outliers.copy()
    
    data = data[data["Time till next peak"] < 7500]
    return data

data = remover_outliers(data)
data.head()

Shape of data : (92927, 12)
Type of data : <class 'pandas.core.frame.DataFrame'>
Total number of outliers : 68341


,Flare number,Solar cycle,Peak counts,Total counts,Energy,X pos,Y pos,Radial,PE,Duration,Time till peak,Time till next peak
0,33,6,52,30840.0,1.0,-637,-216,672,False,196,58.0,772.0
1,34,6,28,37416.0,1.0,-606,-220,645,False,356,134.0,712.0
2,35,6,18,22128.0,1.0,-729,-99,736,False,276,42.0,744.0
3,26,6,36,30168.0,2.0,-538,-219,581,True,244,234.0,1036.0
4,28,6,15,35232.0,1.0,965,-227,992,False,552,134.0,588.0


In [4]:
data.shape

(24581, 12)

In [5]:
data["Time till peak"].min()

2.0

In [6]:
data["Energy"].unique()

array([1., 2., 3., 4., 5.])

## Importe de modelo

In [7]:
import pickle as pk

#Cargar el modelo

with open('modelo_guardar.pkl', 'rb') as modelo:
    modelo_cargado = pk.load(modelo)

In [ ]:
from 

In [8]:
#Valor a predecir:
Flare_number = 1
Solar_cycle = 4
Peak_counts = 1000
Total_counts = 125
Energy = 2
X_pos = 0
Y_pos = 0
Radial = 340
PE = True
Duration = 123
Time_till_peak = 100

dato_a_predecir = pd.DataFrame([[Flare_number, Solar_cycle, Peak_counts, Total_counts, Energy, X_pos, Y_pos, Radial, PE, Duration, Time_till_peak]], columns = ["Flare number", "Solar cycle", "Peak counts", "Total counts", "Energy", "X pos", "Y pos", "Radial", "PE", "Duration", "Time till peak"])
prediccion = modelo_cargado.predict(dato_a_predecir)

dato_a_predecir

,Flare number,Solar cycle,Peak counts,Total counts,Energy,X pos,Y pos,Radial,PE,Duration,Time till peak
0,1,4,1000,125,2,0,0,340,True,123,100


In [9]:
prediccion

array([480.43017506])

In [10]:
def predecirT(Flare_number, Solar_cycle, Peak_counts, Total_counts, Energy, X_pos, Y_pos, Radial, PE, Duration, Time_till_peak):
    dato_a_predecir = pd.DataFrame([[Flare_number, Solar_cycle, Peak_counts, Total_counts, Energy, X_pos, Y_pos, Radial, PE, Duration, Time_till_peak]], columns = ["Flare number", "Solar cycle", "Peak counts", "Total counts", "Energy", "X pos", "Y pos", "Radial", "PE", "Duration", "Time till peak"])
    prediccion = modelo_cargado.predict(dato_a_predecir)
    print("El tiempo estimado para la siguiente llamarada solar son: ", prediccion[0])
    return prediccion[0]

In [11]:
ss =  widgets.Output()

with ss:
    display(HTML("<h1>Modelo de predicción de tiempo entre llamaradas</h1>"))

ss

Output()

In [12]:
#Title =  widgets.Label(value = "Hola mundo", layout = widgets.Layout(width = "auto", justify_content = "center", border = "2px dashed black"))
Title =  widgets.Output()
Title.clear_output()
with Title:
    display(HTML("<h1>Modelo de predicción de tiempo entre llamaradas</h1>"))
Title.layout = {"justify_content" : "center"}


w1 = widgets.IntSlider(value= 30, min = 0, max = 100)
w2 = widgets.IntSlider(value= 3, min = 1, max = 11)
w3 = widgets.IntSlider(value= 200, min = 0, max = 500)
w4 = widgets.IntSlider(value= 200, min = 0, max = 180000)
w5 = widgets.Dropdown(options = [("(3 - 6)", 1), ("(6 - 12)", 2),("(12 - 25)", 3),("(25 - 50)", 4),("(50 - 100)", 5),("(100 - 300)", 6),
                                 ("(300 - 800)", 7),("(800 - 7000)", 8)]) #Cambiar por lista desplegable
w6 = widgets.IntSlider(value= 0, min = -1024, max = 1083)
w7 = widgets.IntSlider(value= 0, min = -625, max = 596)
w8 = widgets.IntSlider(value= 200, min = 0, max = 1110)
w9 = widgets.Checkbox()#Tue o false
w10 = widgets.IntSlider(value= 200, min = 1, max = 900)
w11 = widgets.IntSlider(value= 200, min = 1, max = 450)
ws = [w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11]

n1 =  widgets.Label(value = "Número de llamarada en el dia", layout = widgets.Layout(width = "auto"))
n2 =  widgets.Label(value = "Año del ciclo solar", layout = widgets.Layout(width = "auto"))
n3 =  widgets.Label(value = "Tasa de conteos de radiación por segundo", layout = widgets.Layout(width = "auto"))
n4 =  widgets.Label(value = "Número total de conteos durante la llamarada", layout = widgets.Layout(width = "auto"))
n5 =  widgets.Label(value = "Rango de energia (KeV)", layout = widgets.Layout(width = "auto"))
n6 =  widgets.Label(value = "Posición del destello en segundos de arco desde el centro del sol (X)", layout = widgets.Layout(width = "auto"))
n7 =  widgets.Label(value = "Posición del destello en segundos de arco desde el centro del sol (Y)", layout = widgets.Layout(width = "auto"))
n8 =  widgets.Label(value = "Distancia radial en segundos de arco desde el centro del sol", layout = widgets.Layout(width = "auto"))
n9 =  widgets.Label(value = "Sucedió un evento partículas", layout = widgets.Layout(width = "auto"))
n10 =  widgets.Label(value = "Duración del destello en segundos", layout = widgets.Layout(width = "auto"))
n11 =  widgets.Label(value = "Tiempo hasta el pico en el dia", layout = widgets.Layout(width = "auto"))
ns = [n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11]

tipo = "Es un modelo de: " + str(type(modelo_cargado))
info1 = widgets.Label(value = tipo , layout = widgets.Layout(width = "auto"))
info2 = widgets.Label(value = "Tiene un error de: " , layout = widgets.Layout(width = "auto")) #FALTA POR PONER

info = widgets.VBox([info1,info2])

foto = widgets.Output()
foto.clear_output()
with foto:
    display(Image(filename = "aia_1600_image.jpg"))

vars = {"Flare_number": w1, "Solar_cycle" : w2, "Peak_counts" : w3, "Total_counts": w4, "Energy" :w5, "X_pos": w6,"Y_pos":w7, "Radial": w8,
        "PE": w9, "Duration": w10,"Time_till_peak":w11}

gui2 = interactive_output(predecirT, vars)
gui2.layout = {"border" : "2px dashed red", "justify_content" : "center"}

In [13]:
def graficar_sol(x,y):
    pos_1_x = x + 400
    pos_1_y = y + 400
    pos_2_x = x - 400
    pos_2_y = y - 400
    
    swap_map = sunpy.map.Map(sunpy.data.sample.SWAP_LEVEL1_IMAGE)

    top_right = SkyCoord(pos_1_x * u.arcsec, pos_1_y * u.arcsec, frame=swap_map.coordinate_frame)
    bottom_left = SkyCoord(pos_2_x * u.arcsec, pos_2_y * u.arcsec, frame=swap_map.coordinate_frame)
    swap_submap = swap_map.submap(bottom_left, top_right=top_right)

    fig = plt.figure()
    ax = fig.add_subplot(projection=swap_submap)
    image = swap_submap.plot(axes=ax)
    swap_submap.draw_limb(axes=ax)
    swap_submap.draw_grid(axes=ax)

    # Make some room and put the title at the top of the figure
    ax.set_position([0.1, 0.1, 0.8, 0.7])
    ax.set_title("Gráfico del sol", pad=45)
    
    plt.show()

graf = interactive_output(graficar_sol, {"x" : w6, "y" : w7})

In [17]:
grid = widgets.GridspecLayout(n_rows = 12, n_columns = 3, grid_gap = "2px")
grid[0,:] = Title
grid[1,2] = gui2
grid[2,2] = info
grid[3:,2] = foto
#grid[3:,2] = graf

for i in range(len(ns)):
    grid[i+1,0] = ns[i]
    grid[i+1,1] = ws[i]
     
grid

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', justify_content='center'), outputs=({'out…